In [20]:
import os
import sys

if "google.colab" in sys.modules:
    # Running in Colab

    !git clone https://github.com/pthengtr/kcw-analytics.git
    !cd /content/kcw-analytics && git pull origin main

    from google.colab import drive
    drive.mount("/content/drive")

    BASE_FOLDER = "/content/drive/Shareddrives"
    BASE_FOLDER_GIT = "/content"
else:
    # Running in local Jupyter
    BASE_FOLDER = r"G:\Shared drives"
    BASE_FOLDER_GIT = r"C:\Users\Windows 11\Notebook"

print("Using folder:", BASE_FOLDER)

fatal: destination path 'kcw-analytics' already exists and is not an empty directory.
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 1.16 KiB | 595.00 KiB/s, done.
From https://github.com/pthengtr/kcw-analytics
 * branch            main       -> FETCH_HEAD
   a390f23..91c77d8  main       -> origin/main
Updating a390f23..91c77d8
Fast-forward
 notebooks/30_vat_sales_report_excel.ipynb | 130 ++++++++++++++++--------------
 1 file changed, 70 insertions(+), 60 deletions(-)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using folder: /content/drive/Shareddrives


In [21]:
YEAR = 2026
MONTH = 2

In [22]:
import os
import pandas as pd

def load_csv_folder(folder):
    """
    Load all CSV files from a folder into a dictionary of DataFrames.

    Parameters
    ----------
    folder : str or Path
        Folder path containing CSV files.

    Returns
    -------
    dict[str, pd.DataFrame]
        Dictionary where key = filename, value = DataFrame.
    """

    data = {}

    folder = str(folder)  # ensures Path() also works

    for file in os.listdir(folder):
        if file.lower().endswith(".csv"):
            path = os.path.join(folder, file)

            df = pd.read_csv(
                path,
                dtype={
                    "BCODE": "string",
                    "MOBILE": "string",
                    "BILLNO": "string",
                },
                encoding="utf-8-sig",
                low_memory=False
            )

            data[file] = df
            print(f"Loaded: {file} -> {df.shape}")

    return data

In [23]:
folder = os.path.join(
    BASE_FOLDER,
    "KCW-Data",
    "kcw_analytics",
    "02_staging",
    "VAT_Purchases",
    f"{YEAR}_{MONTH:02d}"
)

data = load_csv_folder(folder)

Loaded: VAT_PURCHASE_2026_02_BOOK_6_0.csv -> (1, 12)
Loaded: VAT_PURCHASE_2026_02_BOOK_5_0.csv -> (12, 12)
Loaded: VAT_PURCHASE_2026_02_BOOK_2_0.csv -> (56, 12)
Loaded: VAT_PURCHASE_2026_02_BOOK_UNKNOWN.csv -> (1, 12)
Loaded: VAT_PURCHASE_2026_02_BOOK_1_0.csv -> (395, 12)


In [36]:
folder = os.path.join(
    BASE_FOLDER,
    "KCW-Data",
    "kcw_analytics",
    "03_curated",
)

data_raw = load_csv_folder(folder)

Loaded: fact_sales_bills_all.csv -> (288387, 49)
Loaded: fact_sales_all.csv -> (769820, 37)
Loaded: dim_product.csv -> (29403, 6)
Loaded: dim_account.csv -> (2360, 4)
Loaded: dim_category.csv -> (36, 2)
Loaded: dim_billtype.csv -> (9, 2)
Loaded: dim_date.csv -> (1827, 8)
Loaded: dim_branch.csv -> (2, 3)


In [24]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter

def export_vat_report(
    df,
    sheet_name,
    title_name,
    wb=None,
    company_name="บริษัท เกียรติชัยอะไหล่ยนต์ 2007 จำกัด (สำนักงานใหญ่)",
    company_address="305 หมู่ 1 ต.ชุมแสง อ.วังจันทร์ จ.ระยอง 21210",
    tax_id="0215560000262",
    overwrite_sheet=True,   # if sheet exists, replace it
):
    # ✅ Safe df check (NO "if df:" allowed)
    if df is None:
        raise ValueError("df is None")
    if isinstance(df, pd.Series):
        raise ValueError("df must be a DataFrame, not a Series")
    if df.empty:
        # still create sheet with headers/title, or just skip
        pass

    # Workbook handling
    if wb is None:
        wb = Workbook()
        # remove default sheet if empty default
        if wb.sheetnames == ["Sheet"]:
            wb.remove(wb["Sheet"])

    # Handle existing sheet name
    if sheet_name in wb.sheetnames:
        if overwrite_sheet:
            wb.remove(wb[sheet_name])
        else:
            # make a unique name
            i = 2
            base = sheet_name
            while f"{base}_{i}" in wb.sheetnames:
                i += 1
            sheet_name = f"{base}_{i}"

    ws = wb.create_sheet(sheet_name)

    # Styles
    bold = Font(bold=True)
    center = Alignment(horizontal="center", vertical="center", wrap_text=True)
    left = Alignment(horizontal="left", vertical="center", wrap_text=True)
    header_fill = PatternFill(start_color="F3DFD2", fill_type="solid")
    thin = Side(style="thin")
    border = Border(left=thin, right=thin, top=thin, bottom=thin)

    # Title / Header area
    ws.merge_cells("A1:H1")
    ws["A1"] = title_name
    ws["A1"].font = Font(bold=True, size=14)
    ws["A1"].alignment = center

    ws.merge_cells("A3:C3"); ws["A3"] = "ชื่อสถานประกอบกิจการ"
    ws.merge_cells("D3:F3"); ws["D3"] = company_name
    ws.merge_cells("A4:C4"); ws["A4"] = "ที่อยู่สถานประกอบกิจการ"
    ws.merge_cells("D4:F4"); ws["D4"] = company_address
    ws.merge_cells("A5:C5"); ws["A5"] = "เลขประจำตัวผู้เสียภาษี"
    ws.merge_cells("D5:F5"); ws["D5"] = tax_id

    for r in [3, 4, 5]:
        ws[f"A{r}"].font = bold
        ws[f"A{r}"].alignment = left
        ws[f"D{r}"].alignment = left

    # Table start
    start_row = 7

    # Write df (header + rows)
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), start_row):
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)
            cell.border = border
            if r_idx == start_row:
                cell.font = bold
                cell.fill = header_fill
                cell.alignment = center
            else:
                cell.alignment = left

    ws.freeze_panes = f"A{start_row+1}"

    # Column widths (safe)
    for c_idx, col_name in enumerate(df.columns, 1):
        series = df[col_name].astype(str).fillna("")
        max_len = max(series.map(len).max(), len(str(col_name)))
        ws.column_dimensions[get_column_letter(c_idx)].width = min(max_len + 2, 60)

    # Money formatting
    money_like = {"มูลค่าสินค้า", "ภาษีมูลค่าเพิ่ม", "ยอดสุทธิ", "AMOUNT", "VAT", "TOTAL"}
    for c_idx, col_name in enumerate(df.columns, 1):
        name = str(col_name).strip()
        if name in money_like or any(k in name.upper() for k in ["AMOUNT", "VAT", "TOTAL"]):
            for r in range(start_row + 1, start_row + len(df) + 1):
                ws.cell(row=r, column=c_idx).number_format = '#,##0.00'

    date_cols = {"วันที่", "BILLDATE"}
    for c_idx, col_name in enumerate(df.columns, 1):
        if str(col_name).strip() in date_cols:
            for r in range(start_row + 1, start_row + len(df) + 1):
                cell = ws.cell(row=r, column=c_idx)
                cell.number_format = "dd/mm/yyyy"   # <-- your short format

    return wb

In [25]:
def clean_excel_text(df):
    return df.apply(
        lambda col: col.str.replace(r'^="|"$', '', regex=True)
        if col.dtype == "object" or str(col.dtype) == "string"
        else col
    )

In [26]:
import pandas as pd

def build_vat_report(
    df,
    rename_map=None,
    column_order=None,
    wb=None,
    sheet_name="REPORT",
    title_name="รายงานภาษี",
    clean_text=True,
    reset_seq=True,
    seq_col="SEQ",
    seq_out_col="ลำดับ",
    add_total_row=True,
    total_sum_cols=("มูลค่าสินค้า", "ภาษีมูลค่าเพิ่ม", "ยอดสุทธิ"),
    total_label_col="เลขที่",
    total_label_text="รวม",
    empty_action="write_headers",   # "write_headers" | "skip"
    **export_kwargs
):
    if df is None:
        raise ValueError("df is None")
    if isinstance(df, pd.Series):
        raise ValueError("df must be a DataFrame, not a Series")

    df = df.copy()

    # optional clean excel ="..."
    if clean_text:
        df = clean_excel_text(df)

    # reset seq (supports either SEQ or ลำดับ)
    if reset_seq:
        df = df.reset_index(drop=True)
        if seq_col in df.columns:
            df[seq_col] = range(1, len(df) + 1)
        elif seq_out_col in df.columns:
            df[seq_out_col] = range(1, len(df) + 1)
        else:
            # create SEQ by default if neither exists
            df[seq_out_col] = range(1, len(df) + 1)

    # rename ONLY if rename_map is given AND it seems applicable
    if rename_map:
        # apply rename only if at least one key exists in df
        if any(k in df.columns for k in rename_map.keys()):
            df = df.rename(columns=rename_map)

    # ✅ EMPTY DF HANDLING (place this BEFORE column_order validation)
    if df.empty:
        if empty_action == "skip":
            return df, wb  # do nothing
        # else: write empty sheet with headers if possible
        if column_order:
            df = pd.DataFrame(columns=list(column_order))
        # export empty header sheet
        wb = export_vat_report(
            df,
            sheet_name=sheet_name,
            title_name=title_name,
            wb=wb,
            **export_kwargs
        )
        return df, wb

    # reorder if provided
    if column_order:
        missing = [c for c in column_order if c not in df.columns]
        if missing:
            raise ValueError(f"Missing columns for column_order: {missing}")
        df = df.loc[:, list(column_order)]

    # --- PATCH: add total row at end (no other logic changes)
    if add_total_row:
        total_row = {c: "" for c in df.columns}

        for c in total_sum_cols:
            if c in df.columns:
                total_row[c] = df[c].sum()

        if total_label_col in df.columns:
            total_row[total_label_col] = total_label_text

        # keep SEQ/ลำดับ blank on total row (common report style)
        if seq_col in total_row:
            total_row[seq_col] = ""
        if seq_out_col in total_row:
            total_row[seq_out_col] = ""

        df = pd.concat([df, pd.DataFrame([total_row])], ignore_index=True)


    # export
    wb = export_vat_report(
        df,
        sheet_name=sheet_name,
        title_name=title_name,
        wb=wb,
        **export_kwargs
    )

    return df, wb

In [27]:
def group_vat_by_date(df):
    df = df.copy()

    # ensure sorted so first/last BILLNO is correct
    df = df.sort_values(["วันที่", "เลขที่"])

    # define aggregation
    agg_dict = {
        "เลขที่": lambda x: f"{x.iloc[0]} - {x.iloc[-1]}",
        "มูลค่าสินค้า": "sum",
        "ภาษีมูลค่าเพิ่ม": "sum",
        "ยอดสุทธิ": "sum"
    }

    # all remaining columns → first
    for col in df.columns:
        if col not in ["วันที่", "เลขที่", "มูลค่าสินค้า", "ภาษีมูลค่าเพิ่ม", "ยอดสุทธิ"]:
            agg_dict[col] = "first"

    df_grouped = (
        df
        .groupby("วันที่", as_index=False)
        .agg(agg_dict)
    )

    # rebuild SEQ
    if "ลำดับ" in df_grouped.columns:
        df_grouped["ลำดับ"] = range(1, len(df_grouped) + 1)

    return df_grouped

In [28]:
df_book_1 = data[f"VAT_PURCHASE_{YEAR}_{MONTH:02d}_BOOK_1_0.csv"].copy()
df_book_2 = data[f"VAT_PURCHASE_{YEAR}_{MONTH:02d}_BOOK_2_0.csv"].copy()
df_book_5 = data[f"VAT_PURCHASE_{YEAR}_{MONTH:02d}_BOOK_5_0.csv"].copy()
df_book_6 = data[f"VAT_PURCHASE_{YEAR}_{MONTH:02d}_BOOK_6_0.csv"].copy()
df_unknown = data[f"VAT_PURCHASE_{YEAR}_{MONTH:02d}_BOOK_UNKNOWN.csv"].copy()

In [29]:
wb = None

In [30]:
column_order = [
    "ลำดับ",
    "วันที่",
    "เลขที่",
    "ชื่อผู้ขายสินค้า",
    "เลขผู้เสียภาษี",
    "มูลค่าสินค้า",
    "ภาษีมูลค่าเพิ่ม",
    "ยอดสุทธิ",
    "รายการสินค้า"
]

In [31]:
rename_map = {
    "SEQ": "ลำดับ",
    "BILLDATE": "วันที่",
    "BILLNO": "เลขที่",
    "ACCTNAME": "ชื่อผู้ขายสินค้า",
    "REMARKS_EXTRACT": "เลขผู้เสียภาษี",
    "BEFORETAX": "มูลค่าสินค้า",
    "TAX": "ภาษีมูลค่าเพิ่ม",
    "AFTERTAX": "ยอดสุทธิ",
    "DETAIL": "รายการสินค้า"
}

df_book_1_report, wb = build_vat_report(
    df_book_1,
    rename_map,
    column_order,
    wb=wb,
    sheet_name="Book 1",
    title_name="รายงานภาษีซื้อ"
)

df_book_2_report, wb = build_vat_report(
    df_book_2,
    rename_map,
    column_order,
    wb=wb,
    sheet_name="Book 2",
    title_name="รายงานภาษีซื้อ"
)

df_book_5_report, wb = build_vat_report(
    df_book_5,
    rename_map,
    column_order,
    wb=wb,
    sheet_name="Book 5",
    title_name="รายงานภาษีซื้อ"
)

df_book_6_report, wb = build_vat_report(
    df_book_6,
    rename_map,
    column_order,
    wb=wb,
    sheet_name="Book 6",
    title_name="รายงานภาษีซื้อ"
)

df_unknown_report, wb = build_vat_report(
    df_unknown,
    rename_map,
    column_order,
    wb=wb,
    sheet_name="Unknown",
    title_name="รายงานภาษีซื้อ"
)


In [32]:
folder = os.path.join(
    BASE_FOLDER,
    "KCW-Data",
    "kcw_analytics",
    "04_outputs",
    "VAT_Purchases_Report",
    "vat_purchases_report_hq.xlsx"
)

# ⭐ create directory if missing
os.makedirs(os.path.dirname(folder), exist_ok=True)

wb.save(folder)